In [1]:
# !pip install category_encoders
# !pip install lightgbm

In [2]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 17px;
</style>

In [3]:
import os
import gc
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
from contextlib import contextmanager
import time

import numpy as np
import pandas as pd
import scipy as sp
import random

import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn import preprocessing
import category_encoders as ce
from sklearn.metrics import mean_squared_error

import torch
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
np.random.seed(0)

from pytorch_tabnet.tab_model import TabNetRegressor ##Import Tabnet 
from sklearn.model_selection import ParameterGrid
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

import os
from pathlib import Path
import os.path as osp

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
data_path = osp.join(os.getcwd(),'Data')
df = pd.read_csv(data_path + '/model.csv')
df = df.drop(['Date','News','Unnamed: 0'],axis=1)

In [5]:
scaled_features = StandardScaler().fit_transform(df.values)

In [6]:
train = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

In [7]:
target = 'cap'
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [8]:
for col in train.columns[train.dtypes == 'float64']:
    train.fillna(train.loc[train_indices, col].mean(), inplace=True)

In [9]:
unused_feat = ['Set']
features = [ col for col in train.columns if col not in unused_feat+[target]] 

In [10]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices].reshape(-1, 1)

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices].reshape(-1, 1)

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices].reshape(-1, 1)

In [11]:
SEED = 42

In [12]:
def create_tabnet(n_a=24,n_d=32, n_steps=5, lr=0.02, gamma=1.5, 
                  n_independent=2, n_shared=2, lambda_sparse=1e-4, 
                  momentum=0.3, clip_value=2.):
    
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        n_independent=n_independent, 
        n_shared=n_shared,        
        lambda_sparse=lambda_sparse,
        momentum=momentum, 
        clip_value=clip_value,
        optimizer_fn=torch.optim.Adam,
        scheduler_params = {"gamma": 0.95,
                         "step_size": 20},
        scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15, verbose = 0
    )
    return model

In [13]:
param_grid = dict(n_d = [24, 32],
                  n_a = [24],
                  n_steps = [3, 4, 5],
                  gamma = [1, 1.5, 2],
                  lambda_sparse = [1e-2, 1e-3, 1e-4],
                  momentum = [0.3, 0.4, 0.5],
                  n_shared = [2],
                  n_independent = [2],
                  clip_value = [2.],     
)

In [14]:
grid = ParameterGrid(param_grid)

In [15]:
search_results = pd.DataFrame() 

In [16]:
num_epochs = 25
patience = 7
batch_size = 128
virtual_batch_size = 32

In [17]:
for params in grid:
    params['n_a'] = params['n_d'] # n_a=n_d always per the paper
    tabnet = create_tabnet()
    tabnet.set_params(**params)
    tabnet.fit(X_train=X_train, y_train=y_train,
               X_valid=X_valid, y_valid=y_valid,
               max_epochs=num_epochs, patience=patience,
               batch_size=batch_size, virtual_batch_size=virtual_batch_size,
              )
    y_pred = tabnet.predict(X_test)    
    t_rmse = mse(y_test, y_pred,squared=False)    
    t_mae = mae(y_test, y_pred)
    
    results = pd.DataFrame([params])
    results['rmse'] = np.round(t_rmse, 3)
    results['mae'] = np.round(t_mae, 3)
    search_results = search_results.append(results)

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used 

In [20]:
eval_path = osp.join(os.getcwd(),'Models')
search_results.to_csv(eval_path + '/Tabnet-experiments.csv')